In [1]:
import os
import re
from datetime import datetime
from dateutil import parser
import requests
from lxml.etree import HTML as html
import pandas as pd

In [2]:
# URL template for accessing spot records
SPOT_URL_TEMPL = 'http://trackleaders.com/spot/%s/%s.js'
USER_AGENT = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'
def parse(s):
    return parser.parse(s.replace('(EEST)',''))
def getSpotData(race, racer_id):
    spot_url = SPOT_URL_TEMPL% (race, racer_id)
    spot_feed = requests.get(spot_url, headers={'User-Agent': USER_AGENT}).text
    latlons=eval(spot_feed.split('LLarray = ')[1].split('; polyline')[0])
    SPOT_TS_RE_TEMPL2 = 'received at:(.*?) <br'
    timestamps = re.findall(SPOT_TS_RE_TEMPL2,
                             spot_feed,
                             re.MULTILINE)
    timestamps=[parse(e) for e in timestamps]
    return latlons,timestamps

In [3]:
race = "transconrace19"
racers = pd.read_pickle('racers.pkl')

Make the output directories. The tracks combined with the timestamps are in the 'DATE directory. Timestamps are only available for the last 600 locations. The complete history without timestamps is in the DTAE/Tracks directory.

In [4]:
dirout = str(datetime.now().date())
if not os.path.exists(dirout):
    os.mkdir(dirout)
if not os.path.exists(dirout+'/Tracks'):
    os.mkdir(dirout+'/Tracks')    

In [ ]:
for racer in racers['Name']:
    try:
        latlons,timestamps = getSpotData(race,racer);
        pd.DataFrame({'LatLon': latlons[len(latlons)-len(timestamps):],'Timestamp':timestamps}).to_pickle(dirout+'/'+racer+'.pkl')
        pd.DataFrame({'LatLon': latlons}).to_pickle(dirout+'/'+'Tracks/'+racer+'.pkl')
    except:
         print(racer)


Just retry the names which are printed. The eror is always some connection problem (time out,...)

In [ ]:
racer = 'Chris_Bower'
latlons,timestamps = getSpotData(race,racer);
pd.DataFrame({'LatLon': latlons[len(latlons)-len(timestamps):],'Timestamp':timestamps}).to_pickle(dirout+'/'+racer+'.pkl')